In [1]:
import os
import json
import numpy as np
import textdistance as td
from openai import OpenAI
import numbers
from dotenv import load_dotenv

load_dotenv()

True

# **Feature 1**

In [2]:
def compare_job_posting(job_description, client, role_stats):
    output = {}
    for key in  ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits", "education"]:
        output[key] = None
        
    format = '''
    {
     "role": "job role"
     "salary": 100000,
     "education": "BS",
     "experience": 5,
     "num_skills": 5,
     "num_responsibilities": 5,
     "num_benefits": 5
    }
    '''
    description = ("where role is the job position or title, "
                   "salary is the estimated salary in dollars, "
                   "ed is the required education, "
                   "exp is the desired years of experience, "
                   "num_skills is number of skills listed, "
                   "num_responsibilities is the number of responsibilities listed, "
                   "and num_benefits is the number of benefits listed.  "
                   "If something is not provided, put a 'null' instead.  "
                   "Do not include any delimiters or special characters "
                   "other than what is needed to load this JSON string into Python.")
    
    is_json = False
    attempts = 0
    while not is_json:
        prompt = "Based off this job description:\n" + job_description + \
        "\nCan you give me a JSON object in this format with this information:\n" + format + description
        response = client.chat.completions.create(
            messages = [{"role": "user", "content": prompt}],
            model = "gpt-3.5-turbo"
        )

        # make sure we don't query too many times
        attempts += 1
        if attempts >= 3:
            return output
            
        # keep looping until we get a valid JSON format
        try:
            parsed_info = json.loads(response.choices[0].message.content)
            role = parsed_info["role"]
            is_json = True
        except:
            pass

    prompt = "Here is a list of job roles: \n" + str(list(role_stats.keys())) + f"\nCan you give me only one role from that list which is most similar to '{parsed_info['role']}'?"
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    # every so often AI will return a role that's not actually in the list
    # if this is the case, find the most similar role in the list to the returned role using cosine similarity
    ratios = []
    for role in list(role_stats.keys()):
        ratios.append(td.cosine(response.choices[0].message.content.lower().strip(), role.lower().strip()))
    stats = role_stats[list(role_stats.keys())[np.argmax(ratios)]]

    tol = .1    
    for key in ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits"]:
        if key not in parsed_info.keys():
            output[key] = None
            
        if isinstance(parsed_info[key], numbers.Number):
            if parsed_info[key] > (1 + tol) * stats[key]:
                output[key] = "above"
            elif parsed_info[key] < (1 - tol) * stats[key]:
                output[key] = "below"
            else:
                output[key] = "average"
    
    if type(parsed_info["education"]) == str:
        if parsed_info["education"][0] == "B":
            ed = 1
        if parsed_info["education"][0] == "M":
            ed = 2
        if parsed_info["education"][0] == "P":
            ed = 3
        else:
            ed = 0
        if ed > (1 + tol) * stats["education"]:
            output["education"] = "above"
        elif ed < (1 - tol) * stats["education"]:
            output["education"] = "below"
        else:
            output["education"] = "average"

    return output

def translate_output(output):
    description = ("where salary is whether or not it is above or below average, "
                   "education is whether or not it is above or below average, "
                   "experience is whether or not it is above or below average, "
                   "num_skills is whether or not it is above or below average, "
                   "num_responsibilities is whether or not it is above or below average, "
                   "and num_benefits is whether or not it is above or below average. "
                   "If something is 'None', it means no information could be found for that attribute.  ")
    prompt = "Here is a raw JSON output object: \n" + str(output) + "\n" + description + \
    "Can you translate this object into a more readable format, intended for a user that wants to compare this job to other similar jobs?"
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    return response.choices[0].message.content

In [3]:
with open("../outputs/role_stats.json") as f:
    role_stats = json.load(f)
    
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

print("-- Assistant --")
print("Please paste the job description here: \n")
print("-- User --")
job_description = input()
output = compare_job_posting(job_description, client, role_stats)
response = translate_output(output)
print("\n-- Assistant --")
print(response)

-- Assistant --
Please paste the job description here: 

-- User --


 Software Developer - AI Trainer (Contract)- job post DataAnnotation 4.4out of 5 New York, NY•Remote From $40 an hour - Contract You must create an Indeed account before continuing to the company website to apply Profile insights Find out how your skills align with the job description Skills  Do you have experience in AI? Education  Do you have a Bachelor's degree? &nbsp; Job details Here’s how the job details align with your profile. Pay From $40 an hour Job type Contract &nbsp; Benefits Pulled from the full job description Work from home &nbsp; Full job description We are looking for proficient programmers to join our team to train our AI chatbots to code. You will work with the chatbots that we are building in order to measure their progress, as well as write and evaluate code.  In this role you will need to be proficient in at least one programming language and able to solve coding problems (think LeetCode, HackerRank, etc). For each coding problem, you must be able to explain how 


-- Assistant --
This job has a salary below average, education below average, no information on experience, number of skills below average, number of responsibilities above average, and number of benefits below average. This information can be used to compare this job to other similar jobs in terms of salary, education level, number of skills, responsibilities, and benefits offered.


# **Feature 2**

In [4]:
def list_job_skills(role_req, client, role_skills):
    skills = []

    prompt = ("Here is a list of job roles: \n" + str(list(role_skills.keys())) + f"\nCan you give me only one role from that list which is most similar to this user request: '{role_req}'?  "
             "Only return the name of the role, nothing else.")
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    # every so often AI will return a role that's not actually in the list
    # if this is the case, find the most similar role in the list to the returned role using cosine similarity
    ratios = []
    for role in list(role_skills.keys()):
        ratios.append(td.cosine(response.choices[0].message.content.lower().strip(), role.lower().strip()))
    skills = role_skills[list(role_skills.keys())[np.argmax(ratios)]]

    return skills

def translate_output(output):
    prompt = "Here is a list of skills for a specific job role: \n" + str(output) + \
    "Can you translate this list into a more readable format, intended for a user that wants to find out what skills are required for this certain job role?  Feel free to expand a little as you see fit."
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    return response.choices[0].message.content

In [5]:
with open("../outputs/role_skills.json") as f:
    role_skills = json.load(f)

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

print("-- Assistant --")
print("What job role are you looking for?\n")
print("-- User --")
role = input()
output = list_job_skills(role, client, role_skills)
response = translate_output(output)
print("\n-- Assistant --")
print(response)

-- Assistant --
What job role are you looking for?

-- User --


 software developer



-- Assistant --
Skills required for this job role include:
1. Frontend and Backend Development 
2. Backend Development 
3. Database Integration 
4. API Knowledge 

Candidates should have a strong understanding of both frontend and backend development processes, with a focus on backend development specifically. Database integration skills are essential for managing data efficiently, and a good grasp of APIs is necessary for seamless communication between different systems.


# **Feature 3**

In [6]:
def get_relevant_jobs(user_request, client, role_stats, role_skills):
    top_roles = []
    
    format = '''
    {
     "skills": ["skill 1", "skill 2"]
     "salary": 100000,
    }
    '''
    description = ("where skills is the skills that the user has said they have, "
                   "and salary is their desired salary in dollars.  "
                   "If something is not provided, put a 'null' instead.  "
                   "Do not include any delimiters or special characters "
                   "other than what is needed to load this JSON string into Python.")
    
    is_json = False
    attempts = 0
    while not is_json:
        prompt = "Based off this description of a user's desires in a job they are searching for:\n" + user_request + \
        "\nCan you give me a JSON object in this format with this information:\n" + format + description
        response = client.chat.completions.create(
            messages = [{"role": "user", "content": prompt}],
            model = "gpt-3.5-turbo"
        )

        # make sure we don't query too many times
        attempts += 1
        if attempts >= 3:
            return output
            
        # keep looping until we get a valid JSON format
        try:
            parsed_info = json.loads(response.choices[0].message.content)
            is_json = True
        except:
            pass

    satisfy_salary = []
    if isinstance(parsed_info["salary"], numbers.Number):
        for role in role_stats.keys():
            if role_stats[role]["salary"] >= parsed_info["salary"]:
                satisfy_salary.append(role)
    
    ratios = []
    for role in satisfy_salary:
        similarities = []
        for role_skill in role_skills[role]:
            max_sim = 0
            for user_skill in parsed_info["skills"]:
                dist = td.cosine(role_skill.lower().strip(), user_skill.lower().strip())
                if dist > max_sim:
                    max_sim = dist
            similarities.append(max_sim)
        ratios.append(len([s for s in similarities if s > .75]) / len(similarities))
    
    top_roles = []
    indices = np.array(ratios).argsort()[-5:][::-1]
    for i in indices:
        if ratios[i] > 0:
            top_roles.append(satisfy_salary[i])

    return top_roles

def translate_output(output):
    if len(output) == 0:
        return "Try changing your search criteria a little bit.  I couldn't find any suitable jobs based off the salary and skills you gave me."
        
    prompt = "Here is a list of job roles that fit the user based on a set of criteria: \n" + str(output) + \
    "Can you translate this list into a more readable format, intended for a user that wants to find what jobs are suitable for them based off their desired salary and the skills they possess?  " + \
    "Feel free to expand a little as you see fit."
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    return response.choices[0].message.content

In [7]:
with open("../outputs/role_stats.json") as f:
    role_stats = json.load(f)
with open("../outputs/role_skills.json") as f:
    role_skills = json.load(f)

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

print("-- Assistant --")
print("Tell me about yourself and what you're looking for in a job.  What sort of salary are you looking for?  What skills do you possess right now?\n")
print("-- User --")
role = input()
output = get_relevant_jobs(role, client, role_stats, role_skills)
response = translate_output(output)
print("\n-- Assistant --")
print(response)

-- Assistant --
Tell me about yourself and what you're looking for in a job.  What sort of salary are you looking for?  What skills do you possess right now?

-- User --


 I want a salary of at least 80K.  I know a lot about machine learning, and also can do some software development.  I've used html, python, java, and javascript, and I also know sql.



-- Assistant --
Based on your desired salary and skills, the following job roles may be suitable for you:

1. Human Resources Director: This role involves overseeing all aspects of human resources, including recruitment, training, and employee relations. It requires strong leadership and communication skills, as well as a deep understanding of labor laws and regulations. The salary for this position is generally high, reflecting the level of responsibility involved.

2. Mechanical Design Engineer: As a mechanical design engineer, you will be responsible for creating and testing new products and equipment. This role requires strong analytical and problem-solving skills, as well as proficiency in computer-aided design (CAD) software. The salary for this position is typically competitive, reflecting the demand for skilled engineers in the industry.

3. Sales Manager: Sales managers are responsible for leading and motivating a team of sales representatives to meet and exceed targets. This